In [1]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
from rnn_classes import Dog, DogInput, Race, Races, GRUNet
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime
from training_testing import validate_model, train_regular, train_log, train_super_batch

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

In [4]:
torch.__version__

'1.12.1'

In [5]:
def model_saver(model, optimizer, epoch, loss, hidden_state_dict, model_name = None):
    
    pathtofolder = "C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model"
    # model_name = wandb.run.name
    if not model_name:
        model_name = "test NZ GRU saver"
    isExist = os.path.exists(
        f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/"
    )
    if isExist:
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optim": optimizer.state_dict(),
                "loss": loss,
                "db":hidden_state_dict,
            },
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{epoch}.pt",
        )
    else:
        print("created path")
        os.makedirs(
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/"
        )
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optim": optimizer.state_dict(),
                "loss": loss,
                "db":hidden_state_dict,
            },
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{epoch}.pt",
        )

In [6]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [7]:
def compare_model_to_bf(model:GRUNet,raceDB:Races,example_ct):
    with torch.no_grad():
        sft_max = nn.Softmax(dim=-1)
        l_sftmax = nn.LogSoftmax(dim=-1)
        nnl_loss = nn.KLDivLoss(reduction='batchmean')
        full_test_races = raceDB.get_test_input(range(0,len(raceDB.test_race_ids)))
        full_test_races_w_prices = []
        excluded, included = 0,0
        for r in full_test_races:
            if 0 in r.prices or -1 in r.prices:
                excluded+=1
            else:
                full_test_races_w_prices.append(r)
                included+=1
        print(included,excluded)

        output = l_sftmax(model(full_test_races_w_prices))
        bf_prices = torch.log(torch.tensor([x.implied_prob for x in full_test_races_w_prices ]).to('cuda:0'))
        full_classes = torch.stack([x.classes for x in full_test_races_w_prices ])

        print()

        print(f"our loss = {nnl_loss(output,full_classes)}")
        print(f"their loss = {nnl_loss(bf_prices ,full_classes)}")
        wandb.log({"our loss":nnl_loss(output,full_classes), "their loss":nnl_loss(bf_prices ,full_classes)})

In [8]:
#Testing
def validate_model(model:GRUNet,raceDB:Races,criterion, batch_size, example_ct, epoch_loss, batch_ct):
    torch.autograd.set_detect_anomaly(True)
    sft_max = nn.Softmax(dim=-1)
    l_sftmax = nn.LogSoftmax(dim=-1)

    batch_size=10
    len_test = len(raceDB.test_race_ids)-batch_size
    list_t = [] 
    last = 0
    loss_val = 0 
    correct = 0
    total = 0
    model.eval()
    actuals = []
    preds = []
    grades = []
    tracks = []
    pred_confs = []
    bfsps = []
    start_prices = []
    loss_l = []
    loss_t = []
    margins_l = []
    preds_l = []
    pred_sftmax = []
    raw_margins = []
    raw_places = []
    margins_prob = []

    prices_list = []
    raw_prices = []

    price_dict = {}
    price_dict['prices'] = []
    price_dict['imp_prob'] = []
    price_dict['pred_prob'] = []
    price_dict['pred_price'] = []
    price_dict['margin'] = []
    price_dict['onehot_win'] = []
    price_dict['raceID'] = []
    price_dict['dogID'] = []
    race_ids = []


    with torch.no_grad():
        # loss_comparison
        # full_test_races = raceDB.get_test_input(range(0,len(raceDB.test_race_ids)))
        # output = l_sftmax(model(full_test_races))
        # bf_prices = l_sftmax(torch.tensor([x.prices for x in full_test_races]).to('cuda:0'))
        # full_classes = torch.stack([x.classes for x in full_test_races])
        
        # print(f"our loss = {nnl_loss(output,full_classes)}")
        # print(f"their loss = {nnl_loss(output,full_classes)}")

        for i in trange(0,len_test,batch_size, leave=False):
            races_idx = range(last,last+batch_size)
            last = i+batch_size
            race = raceDB.get_test_input(races_idx)
            #tracks.extend([r.track for r in race])

            X = race
            y = torch.stack([x.classes for x in race])
            output = model(X)
            race_ids.extend([x.raceid for x in race])
            #print(y)

            _, actual = torch.max(y.data, 1)
            onehot_win = F.one_hot(actual, num_classes=8)
            conf, predicted = torch.max(output.data, 1)
            correct += (predicted == actual).sum().item()

            softmax_preds = sft_max(output)

            
            total += batch_size
            actuals.extend(actual.tolist())
            preds.extend(predicted.tolist())
            pred_confs.extend(conf.tolist())
            tracks.extend([r.track_name for r in race])
            grades.extend([r.grade for r in race])
            for i,dog_idx in enumerate(actual.tolist()):
                bfsps.append(race[i].dogs[dog_idx].bfsp)
                #start_prices.append(race[i].dogs[dog_idx].sp)

            
            loss = criterion(output, y).detach()
            loss_tensor = validation_CLE(output,y)
            loss_t.append(loss_tensor.tolist())

            loss_l.append(loss.tolist())
            preds_l.append(output.tolist())
            pred_sftmax.append(softmax_preds.tolist())
            margins_l.append(y.tolist())
            margins_prob.append(y.tolist())
            raw_margins.append([x.raw_margins for x in race])
            raw_places.append([x.raw_places for x in race])
            loss_val += loss

            price_dict['prices'].extend([x.prices for x in race])
            price_dict['imp_prob'].extend([x.implied_prob for x in race])
            price_dict['pred_prob'].extend(softmax_preds.tolist())
            #print([(1/(x+(-7**10))).tolist() for x in torch.exp(output)])
            price_dict['pred_price'].extend([(1/(x)).tolist() for x in softmax_preds])
            price_dict['margin'].extend([x.raw_margins for x in race])
            price_dict['onehot_win'].extend(onehot_win.tolist())
            price_dict['raceID'].extend([[x.raceid]*8 for x in race])
            price_dict['dogID'].extend([x.list_dog_ids() for x in race])

            

        loss_list = []

        #print("start loss calc")
        for i,l in enumerate(loss_l):
            for j in range(0,7):
                loss_list.append([preds_l[i][j],margins_l[i][j],loss_t[i][j],l[j],pred_sftmax[i][j],margins_prob[i][j], raw_margins[i][j], raw_places[i][j]])

    loss_df = pd.DataFrame(loss_list, columns=['Preds','Margins','Indiviual Losses','Losses','softmaxPreds','Softmax Margins','Raw margins', 'Raw places'])
    loss_table = wandb.Table(dataframe=loss_df)


    logdf = pd.DataFrame(data = {"actuals":actuals, "preds":preds,"conf":pred_confs, "grade":grades, "track":tracks, "bfsps":bfsps})#, "sp":start_prices })
    
    logdf['correct'] = logdf.apply(lambda x: 1 if x['actuals']==x['preds'] else 0, axis=1)
    logdf['profit'] = logdf.apply(lambda x: 0 if x['bfsps']<1 else x['bfsps']-1  if x['correct'] else -1, axis=1)
    logdf.to_csv('logDFtest.csv')

    table = wandb.Table(dataframe=logdf)

    logdf['count'] = 1
    logdf['eligible_ct'] = logdf.apply(lambda x: 1 if x['bfsps'] > 1 else 0, axis = 1)

    track_df = logdf.groupby('track', as_index=False).sum(numeric_only=True).reset_index()
    track_table = wandb.Table(dataframe=track_df)
    
    prices_df = pd.DataFrame(price_dict)


    prices_df['sum_price'] = prices_df.apply(lambda x: sum(x['prices']), axis = 1)
    prices_df = prices_df[prices_df['sum_price']>0]

    prices_flat = [item for sublist in prices_df['prices'].tolist() for item in sublist]
    pred_prices = [item for sublist in prices_df['pred_price'].tolist() for item in sublist]
    onehot_win  = [item for sublist in prices_df['onehot_win'].tolist() for item in sublist]
    flat_margins = [item for sublist in prices_df['margin'].tolist() for item in sublist]
    flat_dogs = [item for sublist in prices_df['dogID'].tolist() for item in sublist]
    flat_races = [item for sublist in prices_df['raceID'].tolist() for item in sublist]
    all_price_df = pd.DataFrame(data={'flat_dogs':flat_dogs,'flat_races':flat_races,'prices':prices_flat, 'pred_price':pred_prices, 'onehot_win':onehot_win,'split_margin':flat_margins})
    all_price_df = all_price_df[all_price_df['prices']>1]
    all_price_df['imp_prob'] =  all_price_df.apply(lambda x: 1/x['prices'], axis = 1)
    all_price_df['pred_prob'] =  all_price_df.apply(lambda x: 1/x['pred_price'], axis = 1)
    all_price_df['bet amount'] = all_price_df.apply(lambda x: (x['pred_prob']-x['imp_prob'])*100 if (x['pred_prob']>x['imp_prob'])&(1>x['imp_prob']>0) else 0, axis = 1)
    all_price_df['profit'] = all_price_df.apply(lambda x: x['bet amount']*(x['prices']-1)*0.95 if x['onehot_win'] else -1*x['bet amount'], axis = 1)
    all_price_df['flat_profit'] = all_price_df.apply(lambda x: 1*(x['prices']-1)*0.95 if (x['onehot_win'] and x['bet amount']) else -1, axis = 1)
    all_price_df['colour'] = all_price_df.apply(lambda x: "profitz" if x['profit']>0 else ("loss" if x['profit']<0 else ("no bet - win" if x['onehot_win'] else "no bet")), axis=1)

    all_price_table = wandb.Table(dataframe=all_price_df)

    all_price_df.to_pickle('all_price_df_nz.npy')


    price_table = wandb.Table(dataframe=prices_df)

    try:
        wandb.log({"table_key": table,"loss_table": loss_table,"track_df": track_table,"price_table":price_table, 'allprice_df':all_price_df })
    except Exception as e:
        print(e)

    #print(f"accuray: {correct/total}")
    wandb.log({"accuracy": correct/total, "loss_val": torch.mean(loss_val)/len_test, "correct": correct, 'profit': logdf[logdf['actuals']==logdf['preds']]['profit'].sum(), 'multibet profit':all_price_df['profit'].sum(), 'multibet outlay':all_price_df['bet amount'].sum() })

    return correct/total, torch.mean(loss_val)

In [9]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
#dog_stats_file = open( 'new gru input 2023-01.npy', 'rb')
hidden_size = 64
raceDB = build_dataset('gru_input.npy', hidden_size ,state_filter="NSW")

(1127562, 17)
(1127562, 17)
(278090, 20)
num_features_per_dog=20


  0%|          | 0/14289 [00:00<?, ?it/s]

c:\Users\Nick\.conda\envs\pytorch\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


  0%|          | 0/39020 [00:00<?, ?it/s]

c:\Users\Nick\.conda\envs\pytorch\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


number of races = 39020, number of unique dogs = 14289


In [10]:
date = datetime.datetime.strptime("2022-12-01", "%Y-%m-%d")
raceDB.create_test_split_date(date)

Train examples 37004, Test examples 2016


In [11]:
# nz_races = raceDB.get_test_input(range(0,len(raceDB.test_race_ids)-1))
# dates = [r.race_date for r in nz_races]
# all_nz_dates = NZ.groupby('date').size()

In [12]:
# dog_stats_file = open( r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\prediction validation 2023-01.npy', 'rb')
# hidden_size = 64
# predDB = build_dataset(dog_stats_file, hidden_size)

In [13]:
# #os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
# dog_stats_file = open( r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\forms\prediction_inputs_gru.npy', 'rb')
# hidden_size = 64
# predDB = build_pred_dataset(dog_stats_file, hidden_size)

In [14]:
# raceDB.create_hidden_states_dict()
# predDB.fill_hidden_states_from_dict(hidden_dict=raceDB.hidden_states_dict)
# # len(raceDB.hidden_states_dict)
# # predDB.to_cuda()

In [15]:
#Testing
def predict_model(model,predDB):
    torch.autograd.set_detect_anomaly(True)
    list_t = [] 
    last = 0
    loss_val = 0 
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():

        races_idx = range(0,len(predDB.raceIDs)-1)
        race = predDB.get_race_input(races_idx)
        X = race
        # for i,r in enumerate(race):
        #     print(r.raceid, r.track_name)
        #     #print(i,r.lstm_input())

        output = model(X)
        
        print(output)

        _, predicted = torch.max(output.data, 1)

        for i,r in enumerate(race):
            print(r.raceid, r.track_name, r.dogs[predicted[i].item()])

        print(predicted)



In [16]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="GRU - FastTrack - AUS Testing", config=config):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss", summary="min")
      wandb.define_metric("test_accuracy", summary="max")
      wandb.define_metric("bfprofit", summary="max")
      wandb.define_metric("multibet profit", summary="max")
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy

      model = GRUNet(input_size,config['hidden_size'])
      if model_state_dict:
        model.load_state_dict(model_state_dict)
      criterion = nn.CrossEntropyLoss(reduction='none') #, label_smoothing=0.1 )
      optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

      scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',threshold=0.0001, patience=50, verbose=True, factor=0.5)
      model = model.to(device)
      # optimizer = optimizer.to(device)
      print(model)

      # and use them to train the model
      try:
        train_super_batch(model, dataset, criterion, optimizer, scheduler, config)
      except KeyboardInterrupt:
        print("finished Early")
      dataset.create_hidden_states_dict()
      if sweep:
        raceDB.reset_all_lstm_states
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [17]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':hidden_size,'batch_size': 100, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64 , 'learning_rate': 0.001, 'loss': 'L1', 'l1_beta':0.1,  'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1,'batch_before_backwards':25}

In [18]:
# raceDB.detach_all_hidden_states()
# raceDB.create_test_split()
# raceDB.reset_all_lstm_states()

In [19]:
# saved_dict = torch.load(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\savedmodel\long nsw 2000 RUN\long nsw 2000 RUN_450.pt")
# model_state_dict = saved_dict['model_state_dict']
# raceDB.fill_hidden_states_from_dict(saved_dict['db'])
# raceDB.to_cuda()
# # # predDB.fill_hidden_states_from_dict(hidden_dict=raceDB.hidden_states_dict)
# # # predDB.to_cuda()

In [20]:
raceDB.detach_all_hidden_states()

In [21]:
#(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static)

(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly (use `wandb login --relogin` to force relogin)


{'hidden_size': 64, 'batch_size': 100, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.001, 'loss': 'L1', 'l1_beta': 0.1, 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1, 'batch_before_backwards': 25}
1000
{'hidden_size': 64, 'batch_size': 100, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.001, 'loss': 'L1', 'l1_beta': 0.1, 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1, 'batch_before_backwards': 25}
GRUNet(
  (gru1): GRUCell(232, 64)
  (gru2): GRUCell(232, 64)
  (gru3): GRUCell(232, 64)
  (gru4): GRUCell(232, 64)
  (gru5): GRUCell(232, 64)
  (gru6): GRUCell(232, 64)
  (gru7): GRUCell(232, 64)
  (gru8): GRUCell(232, 64)
  (rl0): ReLU()
  (drop0): Dropout(p=0.3, inplace=False)
  (fc0): Linear(in_features=512, out_features=512, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=512, out_features=64, bias=True)
  (rl2): R

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/201 [00:00<?, ?it/s]

  0%|          | 0/201 [00:00<?, ?it/s]

...

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmp9jp6xfydwandb-media\\2jjc418o.table.json'


  0%|          | 0/201 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmp9jp6xfydwandb-media\\8x7ljqbm.table.json'


  0%|          | 0/201 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmp9jp6xfydwandb-media\\mnytigtz.table.json'


In [ ]:

stop
model = GRUNet(203,64).to(device)
model.load_state_dict(model_state_dict)

NameError: name 'stop' is not defined

In [ ]:

dataset.create_hidden_states_dict()

NameError: name 'dataset' is not defined

In [ ]:
sweep_config = {"method": "random"}

metric = {"name": "profit", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "f1_layer_size": {"values": [256]},
    "f2_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "len_data": {"value": len(raceDB.raceIDs)},
    "hidden_size": {"value":64}
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [250]},
        "validation_split": {"value": 0.1},
        "loss": {
            "values": [ "CEL"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            # 'value': 'l1_custom'
        },
        "num_layers": {"values": [2]},
    }
)

parameters_dict.update(
    {
        "learning_rate": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0.00001,
            "max": 0.001,
        },
        "l1_beta": {"value": 0.1
        },
        "batch_size": {
            'values': [25,50,75,100,1000]
        },
        "batch_before_backwards": {
            'values': [3,5,10,15,20,25,30,50,75,100]
        }
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'profit'},
 'parameters': {'batch_before_backwards': {'values': [3,
                                                      5,
                                                      10,
                                                      15,
                                                      20,
                                                      25,
                                                      30,
                                                      50,
                                                      75,
                                                      100]},
                'batch_size': {'values': [25, 50, 75, 100, 1000]},
                'dropout': {'values': [0.3]},
                'epochs': {'values': [250]},
                'f1_layer_size': {'values': [256]},
                'f2_layer_size': {'values': [64]},
                'hidden_size': {'value': 64},
                'l1_beta': {'value': 0.1

{'method': 'random',
 'metric': {'name': 'profit', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adamW'},
  'f1_layer_size': {'values': [256]},
  'f2_layer_size': {'values': [64]},
  'dropout': {'values': [0.3]},
  'len_data': {'value': 39020},
  'hidden_size': {'value': 64},
  'epochs': {'values': [250]},
  'validation_split': {'value': 0.1},
  'loss': {'values': ['CEL']},
  'num_layers': {'values': [2]},
  'learning_rate': {'distribution': 'uniform', 'min': 1e-05, 'max': 0.001},
  'l1_beta': {'value': 0.1},
  'batch_size': {'values': [25, 50, 75, 100, 1000]},
  'batch_before_backwards': {'values': [3,
    5,
    10,
    15,
    20,
    25,
    30,
    50,
    75,
    100]}}}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="LSTM_sweeps")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: mng6fiip
Sweep URL: https://wandb.ai/nickojelly/LSTM_sweeps/sweeps/mng6fiip


wandb: Agent Starting Run: 4qf02z0o with config:
wandb: 	batch_before_backwards: 5
wandb: 	batch_size: 50
wandb: 	dropout: 0.3
wandb: 	epochs: 250
wandb: 	f1_layer_size: 256
wandb: 	f2_layer_size: 64
wandb: 	hidden_size: 64
wandb: 	l1_beta: 0.1
wandb: 	learning_rate: 0.0009437005664695158
wandb: 	len_data: 39020
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_before_backwards': 5, 'batch_size': 50, 'dropout': 0.3, 'epochs': 250, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 64, 'l1_beta': 0.1, 'learning_rate': 0.0009437005664695158, 'len_data': 39020, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
250
{'batch_before_backwards': 5, 'batch_size': 50, 'dropout': 0.3, 'epochs': 250, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 64, 'l1_beta': 0.1, 'learning_rate': 0.0009437005664695158, 'len_data': 39020, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
GRUNet(
  (gru1): GRUCell(232, 64)
  (gru2): GRUCell(232, 64)
  (gru3): GRUCell(232, 64)
  (gru4): GRUCell(232, 64)
  (gru5): GRUCell(232, 64)
  (gru6): GRUCell(232, 64)
  (gru7): GRUCell(232, 64)
  (gru8): GRUCell(232, 64)
  (rl0): ReLU()
  (drop0): Dropout(p=0.3, inplace=False)
  (fc0): Linear(in_features=512, out_features=512, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1

  0%|          | 0/250 [00:00<?, ?it/s]

...

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmpy317t0jdwandb-media\\e45of5qk.table.json'


  0%|          | 0/313 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmpy317t0jdwandb-media\\2gmddvs4.table.json'


  0%|          | 0/313 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmpy317t0jdwandb-media\\2l2pmwgt.table.json'
